In [1]:
import tensorflow as tf
tf.keras.backend.clear_session()


In [2]:
from tensorflow.keras import layers

class Linear(layers.Layer):
    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(initial_value=w_init(shape=(input_dim, units),
                                                  dtype='float32'),
                             trainable=True)
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(initial_value=b_init(shape=(units,),
                                                  dtype='float32'),
                            trainable=True)
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b
x = tf.ones((2, 2))
print(x.numpy())
linear_layer = Linear(4, 2)
y = linear_layer(x)
print(y)

[[1. 1.]
 [1. 1.]]
tf.Tensor(
[[0.09333631 0.02213606 0.1052626  0.04177345]
 [0.09333631 0.02213606 0.1052626  0.04177345]], shape=(2, 4), dtype=float32)


In [3]:
assert linear_layer.weights == [linear_layer.w, linear_layer.b]


In [4]:
# add_weight
class Linear2(layers.Layer):
    def __init__(self, units=32, input_dim=32):
        super(Linear2, self).__init__()
        self.w = self.add_weight(shape=(input_dim, units),
                                initializer='random_normal',
                                trainable=True)
        self.b = self.add_weight(shape=(units,),
                                initializer='zeros',
                                trainable=True)
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b
x = tf.ones((2, 2))
linear_layer2 = Linear2(4, 2)
y = linear_layer2(x)
print(y)

tf.Tensor(
[[-0.11441192  0.07693142 -0.03292878 -0.05759429]
 [-0.11441192  0.07693142 -0.03292878 -0.05759429]], shape=(2, 4), dtype=float32)


In [5]:
class ComputeSum(layers.Layer):
    def __init__(self, input_dim):
        super(ComputeSum, self).__init__()
        self.total = tf.Variable(initial_value=tf.zeros((input_dim,)),
                                 trainable=False)
    def call(self, inputs):
        self.total.assign_add(tf.reduce_sum(inputs, axis=1))
        return self.total
x = tf.ones((3, 2))
my_sum = ComputeSum(3)
y = my_sum(x)
print(y.numpy())

[2. 2. 2.]


In [6]:
print('weights:', len(my_sum.weights))
print('non-trainable weights:', len(my_sum.non_trainable_weights))

# It's not included in the trainable weights:
print('trainable_weights:', my_sum.trainable_weights)

weights: 1
non-trainable weights: 1
trainable_weights: []


In [21]:
class Linear3(layers.Layer):
    def __init__(self, units=32, **kwargs):
        super(Linear3, self).__init__(**kwargs)
        self.units = units
        
    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal',
                                 trainable=True)
        self.b = self.add_weight(shape=(self.units,),
                                 initializer='random_normal',
                                 trainable=True)
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b
    
    def get_config(self):
        config = super(Linear3, self).get_config()
        config.update({'units': self.units})
        return config

x = tf.ones((3, 2))
linear_layer3 = Linear3(4)
y = linear_layer3(x)
print(y)

config = linear_layer3.get_config()
print(config)
new_layer = Linear3.from_config(config)
y2 = new_layer(x)
print(y2)

tf.Tensor(
[[ 0.08306668  0.05251841 -0.02629673  0.00645482]
 [ 0.08306668  0.05251841 -0.02629673  0.00645482]
 [ 0.08306668  0.05251841 -0.02629673  0.00645482]], shape=(3, 4), dtype=float32)
{'name': 'linear3_14', 'trainable': True, 'dtype': 'float32', 'units': 4}
tf.Tensor(
[[-0.11169399  0.01934814  0.19279274 -0.08042584]
 [-0.11169399  0.01934814  0.19279274 -0.08042584]
 [-0.11169399  0.01934814  0.19279274 -0.08042584]], shape=(3, 4), dtype=float32)


In [8]:
#Layer are recursively composable
class MLPBlock(layers.Layer):
    def __init__(self):
        super(MLPBlock, self).__init__()
        self.linear1 = Linear3(32)
        self.linear2 = Linear3(32)
        self.linear3 = Linear3(1)
    def call(self, inputs):
        x = self.linear1(inputs)
        x = tf.nn.relu(x)
        x = self.linear2(x)
        x = tf.nn.relu(x)
        return self.linear3(x)

x = tf.ones(shape=(3, 64))
mlp = MLPBlock()
y = mlp(x)
print(y)
print('weights:', len(mlp.weights))
print('trainable weights:', len(mlp.trainable_weights))

tf.Tensor(
[[-0.0483809]
 [-0.0483809]
 [-0.0483809]], shape=(3, 1), dtype=float32)
weights: 6
trainable weights: 6


In [9]:
#A layer that creates an activity regularization loss
class ActivityRegularizationLayer(layers.Layer):
    def __init__(self, rate=1e-2):
        super(ActivityRegularizationLayer, self).__init__()
        self.rate = rate
    
    def call(self, inputs):
        self.add_loss(self.rate * tf.reduce_sum(inputs))
        return inputs

In [14]:
class OuterLayer(layers.Layer):
    def __init__(self):
        super(OuterLayer, self).__init__()
        
    def build(self, input_shape):
        self.dense = layers.Dense(32, kernel_regularizer=tf.keras.regularizers.l2(1e-3))
        
    def call(self, inputs):
        return self.dense(inputs)
    
layer = OuterLayer()
y = layer(tf.zeros((1, 1)))
print(y)
print(layer.losses)

tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]], shape=(1, 32), dtype=float32)
[<tf.Tensor: shape=(), dtype=float32, numpy=0.0018426968>]


In [ ]:
# Privileged training argument in the call method
class CustomDropout(layers.Layer):
    def __init__(self, rate, **kwargs):
        super(CustomdDropout, self).__init__(**kwargs)
        self.rate = rate
    
    def call(self, inputs, training=None):
        if training:
            return tf.nn.dropout(inputs, rate=self.rate)
        return inputs
    